In [1]:
import folium
import requests
import json
import geopandas as gpd
from pprint import pprint

In [2]:
stations = gpd.read_file('./data/Washington_DC_US/Metro_Stations_Regional/Metro_Stations_Regional.shp')
districts = gpd.read_file('./data/Washington_DC_US/kx-washington-dc-census-tracts-2010-SHP/washington-dc-census-tracts-2010.shp')

In [3]:
districts['pop_dens'] = districts['Total_Popu'] / districts['Acres']
districts.columns

Index(['Not_Hispan', 'Not_Hisp_1', 'Not_Hisp_2', 'Pop_of_1_r', 'Occupied_H',
       'Median_Fed', 'Pop_of_1_1', 'Acres', 'Not_Hisp_3', 'Total_Hisp',
       'Not_Hisp_4', 'Not_Hisp_5', 'Pop_2_or_m', '18_Pop_2_o', 'Not_Hisp_6',
       'Pop_of_1_2', 'Pop_of_1_3', 'Not_Hisp_7', 'Median_F_1', 'Not_Hisp_8',
       'Total_Fede', 'Total_Fe_1', '18_Pop_1_r', '18_Pop_1_1', 'Not_Hisp_9',
       '18_Pop_1_2', 'Total_Fe_2', 'Total_Hous', 'Pop_2_or_1', 'Tract_Numb',
       'Pop_2_or_2', 'Not_Hisp10', 'Total_Fe_3', 'Pop_2_or_3', 'Pop_of_1_4',
       'SHAPE_Leng', 'Not_Hisp11', 'Not_Hisp12', 'Median_F_2', 'Non-Minori',
       'Total_Fe_4', 'Not_Hisp13', '18_Pop_1_3', 'Square_Mil', 'Total_Fe_5',
       'GEOID', 'Total_Non-', 'Not_Hisp14', 'Total_Popu', '18_Pop_2_1',
       'Not_Hisp15', 'Pop_of_1_5', 'Vacant_Hou', '18_Pop_2_2', '18_Pop_2_3',
       'Not_Hisp16', 'Total_Pop_', 'Total_Po_1', '18_Pop_1_4', 'Hispanic_1',
       'Median_F_3', 'Median_F_4', 'Not_Hisp17', '18_Pop_1_5', 'Median_F_5',
       'g

In [4]:
districts.to_crs(4326, inplace=True)
stations.to_crs(4326, inplace=True)

In [5]:
map = folium.Map(location=[38.895,-77.036389], zoom_start=11, tiles='CartoDB Positron')
# folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.Choropleth(
    geo_data=districts['geometry'],  
    data=districts['pop_dens'], 
    columns=['geometry', 'pop_dens'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2, 
    legend_name="Population Density"
).add_to(map)
# folium.GeoJson(stations).add_to(map)
map

In [6]:
districts.sort_values(by='pop_dens',ascending=False, inplace=True)
filtered_dists = districts[0:30]

In [7]:
filtered_stations = stations.sjoin(filtered_dists, predicate='within')

In [8]:
map = folium.Map(location=[38.895,-77.036389], zoom_start=12, tiles='CartoDB Positron')
# folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.Choropleth(
    geo_data=filtered_dists['geometry'],  
    data=filtered_dists['pop_dens'], 
    columns=['geometry', 'pop_dens'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=1.0, 
    line_opacity=0.2, 
    legend_name="Population Density"
).add_to(map)
folium.GeoJson(filtered_stations).add_to(map)
bounds = filtered_dists.total_bounds
map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
map

In [9]:
filtered_center_points = list(zip(*[filtered_stations.geometry.y, filtered_stations.geometry.x]))
filtered_center_points

[(38.92784575919844, -77.03255440732185),
 (38.909605845165814, -77.04341665160557),
 (38.90070579477101, -77.0502800325331),
 (38.90644460270957, -77.02191666537722)]

In [10]:
# url = 'http://localhost:8086/isomap'
url = 'http://135.234.212.65:8086/isomap'
data = {
    'center_points': filtered_center_points, 
    'networktype': 'walk', 
    'travel_time': 15
}
payload = {
    "data": data, 
    "speeds": {
        "walk": 4500,
        "bike": 18000,
        "drive": 60000
    }
}

In [11]:
response = requests.post(url, json=payload)
if response.status_code == 200: 
    print("Request was successful. ")
    result = response.json()
else: 
    print(f"Failed to retreive response. Response message: {response.status_code}")

Request was successful. 


In [12]:
map = folium.Map(location=[38.895,-77.036389], zoom_start=13, tiles='CartoDB Positron')
folium.Choropleth(
    geo_data=districts['geometry'],  
    data=districts['pop_dens'], 
    columns=['geometry', 'pop_dens'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2, 
    legend_name="Population Density"
).add_to(map)
folium.GeoJson(filtered_dists, 
              style_function=lambda feature: {
                  'fillColor': 'white', 
                  'color': 'black', 
                  'weight': 1.5,
                  'fillOpacity': 0
              }).add_to(map)
folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.GeoJson(filtered_stations).add_to(map)
bounds = filtered_dists.total_bounds
map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
map

In [13]:
# v211 took 1006.55 secs to finish
# v213 took 375.85 secs to finish
isomap = gpd.read_file(result['isochrones'])
isomap.to_file('./results/isomap_metro_stations_30_census_tract_highest_popdens_Washington_DC_US.shp')
map.save('./results/isomap_metro_stations_30_census_tract_highest_popdens_Washington_DC_US.html')

In [29]:
filtered_dists2 = districts.merge(districts.sjoin(stations).groupby('Tract_Numb').size().rename('station_count').reset_index())
filtered_dists2.sort_values(by='station_count', ascending=False, inplace=True)
# filtered_dists2 = filtered_dists2[0:3]

In [15]:
filtered_stations2 = stations.clip(filtered_dists2)

In [30]:
map = folium.Map(location=[38.895,-77.036389], zoom_start=13)
# folium.GeoJson(json.loads(result['isochrones'])).add_to(map)
folium.Choropleth(
    geo_data=filtered_dists2['geometry'],  
    data=filtered_dists2['station_count'], 
    columns=['geometry', 'station_count'], 
    key_on='feature.id', 
    fill_color='YlGnBu', 
    fill_opacity=1.0, 
    line_opacity=0.2, 
    legend_name="Station Count in Census Tract"
).add_to(map)
folium.GeoJson(filtered_stations2).add_to(map)
bounds = filtered_dists2.total_bounds
map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
map

NameError: name 'filtered_stations2' is not defined